<a href="https://colab.research.google.com/github/vaishak-krishnan/Kidney-CT-Project/blob/main/KidneyCT_Validation_on_Deeplearning_Architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow pandas scikit-learn

In [ ]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Load dataset
kidney_data = pd.read_csv("E:/Workspace/Final Project/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/kidneyData.csv")

# Define constants
IMG_SIZE = (224, 224)  # Standard image size
BATCH_SIZE = 32

# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(kidney_data, test_size=0.2, stratify=kidney_data['Class'], random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, stratify=train_data['Class'], random_state=42)  # 60% train, 20% val, 20% test

# Initialize data augmentation and rescaling generators
train_datagen = ImageDataGenerator(
    rescale=1./255,                # Normalize pixel values to [0,1]
    horizontal_flip=True,           # Randomly flip images horizontally
    rotation_range=15,              # Rotate images by up to 15 degrees
    zoom_range=0.1                  # Zoom in on images by up to 10%
)

val_test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale validation and test images

# Create data generators for training, validation, and test sets
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='path',                   # Path to images
    y_col='Class',                  # Class labels
    target_size=IMG_SIZE,           # Resize images to IMG_SIZE
    batch_size=BATCH_SIZE,
    class_mode='categorical'        # Use categorical labels for multi-class classification
)

val_generator = val_test_datagen.flow_from_dataframe(
    val_data,
    x_col='path',
    y_col='Class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_dataframe(
    test_data,
    x_col='path',
    y_col='Class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Print summary of data
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Test samples: {test_generator.samples}")


Found 7467 validated image filenames belonging to 4 classes.
Found 2489 validated image filenames belonging to 4 classes.
Found 2490 validated image filenames belonging to 4 classes.
Training samples: 7467
Validation samples: 2489
Test samples: 2490


In [ ]:
import pandas as pd

# Assuming kidney_data is the loaded dataset
# Summary of dataset
print("Dataset Overview")
print("----------------")
print("Total images:", len(kidney_data))

# Display class distribution
class_distribution = kidney_data['Class'].value_counts()
print("\nClass Distribution:")
print(class_distribution)

# Summary of preprocessing
print("\nData Preprocessing:")
print("1. Resized all images to 224x224.")
print("2. Applied data augmentation: horizontal flip, rotation (15 degrees), and zoom (10%).")
print("3. Normalized pixel values to [0, 1] range.")


Dataset Overview
----------------
Total images: 12446

Class Distribution:
Class
Normal    5077
Cyst      3709
Tumor     2283
Stone     1377
Name: count, dtype: int64

Data Preprocessing:
1. Resized all images to 224x224.
2. Applied data augmentation: horizontal flip, rotation (15 degrees), and zoom (10%).
3. Normalized pixel values to [0, 1] range.


In [ ]:
from sklearn.metrics import classification_report


In [ ]:
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Assuming train_generator, val_generator, and test_generator are already defined
# Define and compile VGG16 model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16_base.trainable = False  # Freeze base layers

def build_model(base_model):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes in the output layer
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

vgg16_model = build_model(vgg16_base)

# Train the model
vgg16_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = vgg16_model.evaluate(test_generator, verbose=0)
y_true = test_generator.classes
y_pred = vgg16_model.predict(test_generator, verbose=0).argmax(axis=1)

# Calculate classification metrics
report = classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys()), output_dict=True)
metrics_df = pd.DataFrame(report).transpose()
metrics_df['loss'] = test_loss  # Add test loss to the table

# Display results
print(f"VGG16 Model Test Accuracy: {test_accuracy:.4f}")
print(f"VGG16 Model Test Loss: {test_loss:.4f}")
print(metrics_df[['precision', 'recall', 'f1-score', 'support', 'loss']])


Epoch 1/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 931s 4s/step - accuracy: 0.5637 - loss: 1.0917 - val_accuracy: 0.7348 - val_loss: 0.7452
Epoch 2/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 985s 4s/step - accuracy: 0.7384 - loss: 0.7298 - val_accuracy: 0.7670 - val_loss: 0.5809
Epoch 3/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 1008s 4s/step - accuracy: 0.8043 - loss: 0.5534 - val_accuracy: 0.8108 - val_loss: 0.5297
Epoch 4/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 989s 4s/step - accuracy: 0.8305 - loss: 0.4771 - val_accuracy: 0.8501 - val_loss: 0.4278
Epoch 5/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 1004s 4s/step - accuracy: 0.8613 - loss: 0.4023 - val_accuracy: 0.8634 - val_loss: 0.4076
VGG16 Model Test Accuracy: 0.8687
VGG16 Model Test Loss: 0.3970
              precision    recall  f1-score      support      loss
Cyst           0.285239  0.346361  0.312842   742.000000  0.397031
Normal         0.402764  0.401575  0.402169  1016.000000  0.397031
Stone          0.093750  0.087273  0.090395   275.000000  0.397031
Tumor          0.209375  0

In [ ]:
import pandas as pd
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import classification_report

# Define and compile VGG19 model
vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg19_base.trainable = False  # Freeze base layers

def build_model(base_model):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(len(train_generator.class_indices), activation='softmax')  # Use number of classes in output layer
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

vgg19_model = build_model(vgg19_base)

# Train the model
vgg19_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    verbose=1
)

# Evaluate and calculate metrics
test_loss, test_accuracy = vgg19_model.evaluate(test_generator, verbose=0)
y_true = test_generator.classes
y_pred = vgg19_model.predict(test_generator, verbose=0).argmax(axis=1)

# Classification report for metrics
report = classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys()), output_dict=True)
metrics_df = pd.DataFrame(report).transpose()

# Display results
print(f"VGG19 Model Test Accuracy: {test_accuracy:.4f}")
print(metrics_df[['precision', 'recall', 'f1-score']])


Epoch 1/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 1106s 5s/step - accuracy: 0.5200 - loss: 1.1721 - val_accuracy: 0.6826 - val_loss: 0.8779
Epoch 2/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 1066s 5s/step - accuracy: 0.6975 - loss: 0.8319 - val_accuracy: 0.7131 - val_loss: 0.7503
Epoch 3/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 1051s 4s/step - accuracy: 0.7442 - loss: 0.6875 - val_accuracy: 0.7730 - val_loss: 0.6052
Epoch 4/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 1053s 4s/step - accuracy: 0.7822 - loss: 0.5918 - val_accuracy: 0.7971 - val_loss: 0.5317
Epoch 5/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 1057s 5s/step - accuracy: 0.8121 - loss: 0.5185 - val_accuracy: 0.7991 - val_loss: 0.5246
VGG19 Model Test Accuracy: 0.8004
              precision    recall  f1-score
Cyst           0.305785  0.299191  0.302452
Normal         0.415902  0.535433  0.468158
Stone          0.123457  0.036364  0.056180
Tumor          0.178667  0.146608  0.161058
accuracy       0.338554  0.338554  0.338554
macro avg      0.255953  0.254399  0.246962
weighted avg 

In [ ]:
from tensorflow.keras.applications import ResNet50

# Define and compile ResNet50 model
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50_base.trainable = False  # Freeze base layers

resnet50_model = build_model(resnet50_base)

# Train the model
resnet50_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    verbose=1
)

# Evaluate and calculate metrics
test_loss, test_accuracy = resnet50_model.evaluate(test_generator, verbose=0)
y_true = test_generator.classes
y_pred = resnet50_model.predict(test_generator, verbose=0).argmax(axis=1)

# Classification report for metrics
report = classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys()), output_dict=True)
metrics_df = pd.DataFrame(report).transpose()

# Display results
print(f"ResNet50 Model Test Accuracy: {test_accuracy:.4f}")
print(metrics_df[['precision', 'recall', 'f1-score']])


Epoch 1/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 392s 2s/step - accuracy: 0.4493 - loss: 1.2589 - val_accuracy: 0.6123 - val_loss: 0.9907
Epoch 2/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 384s 2s/step - accuracy: 0.6032 - loss: 0.9945 - val_accuracy: 0.6714 - val_loss: 0.8678
Epoch 3/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 395s 2s/step - accuracy: 0.6343 - loss: 0.9112 - val_accuracy: 0.6304 - val_loss: 0.8951
Epoch 4/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 387s 2s/step - accuracy: 0.6433 - loss: 0.8751 - val_accuracy: 0.5745 - val_loss: 1.0074
Epoch 5/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 384s 2s/step - accuracy: 0.6667 - loss: 0.8000 - val_accuracy: 0.7107 - val_loss: 0.7688
ResNet50 Model Test Accuracy: 0.7129
              precision    recall  f1-score
Cyst           0.289805  0.340970  0.313313
Normal         0.398671  0.472441  0.432432
Stone          0.000000  0.000000  0.000000
Tumor          0.188725  0.168490  0.178035
accuracy       0.325301  0.325301  0.325301
macro avg      0.219300  0.245475  0.230945
weighted avg   

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

# Define and compile EfficientNetB0 model
efficientnetb0_base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
efficientnetb0_base.trainable = False  # Freeze base layers

efficientnetb0_model = build_model(efficientnetb0_base)

# Train the model
efficientnetb0_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    verbose=1
)

# Evaluate and calculate metrics
test_loss, test_accuracy = efficientnetb0_model.evaluate(test_generator, verbose=0)
y_true = test_generator.classes
y_pred = efficientnetb0_model.predict(test_generator, verbose=0).argmax(axis=1)

# Classification report for metrics
report = classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys()), output_dict=True)
metrics_df = pd.DataFrame(report).transpose()

# Display results
print(f"EfficientNetB0 Model Test Accuracy: {test_accuracy:.4f}")
print(metrics_df[['precision', 'recall', 'f1-score']])


Epoch 1/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 241s 986ms/step - accuracy: 0.4073 - loss: 1.2918 - val_accuracy: 0.4078 - val_loss: 1.2958
Epoch 2/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 231s 975ms/step - accuracy: 0.4075 - loss: 1.2851 - val_accuracy: 0.4078 - val_loss: 1.2877
Epoch 3/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 231s 980ms/step - accuracy: 0.4039 - loss: 1.2812 - val_accuracy: 0.4078 - val_loss: 1.2840
Epoch 4/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 230s 972ms/step - accuracy: 0.4028 - loss: 1.2864 - val_accuracy: 0.4078 - val_loss: 1.2822
Epoch 5/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 230s 973ms/step - accuracy: 0.4100 - loss: 1.2792 - val_accuracy: 0.4078 - val_loss: 1.2833
EfficientNetB0 Model Test Accuracy: 0.4080
              precision    recall  f1-score
Cyst           0.000000  0.000000  0.000000
Normal         0.408032  1.000000  0.579578
Stone          0.000000  0.000000  0.000000
Tumor          0.000000  0.000000  0.000000
accuracy       0.408032  0.408032  0.408032
macro avg      0.102008  0.250000  0.1

C:\Program Files\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Program Files\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Program Files\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from tensorflow.keras.applications import DenseNet121

# Define and compile DenseNet121 model
densenet121_base = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
densenet121_base.trainable = False  # Freeze base layers

densenet121_model = build_model(densenet121_base)

# Train the model
densenet121_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    verbose=1
)

# Evaluate and calculate metrics
test_loss, test_accuracy = densenet121_model.evaluate(test_generator, verbose=0)
y_true = test_generator.classes
y_pred = densenet121_model.predict(test_generator, verbose=0).argmax(axis=1)

# Classification report for metrics
report = classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys()), output_dict=True)
metrics_df = pd.DataFrame(report).transpose()

# Display results
print(f"DenseNet121 Model Test Accuracy: {test_accuracy:.4f}")
print(metrics_df[['precision', 'recall', 'f1-score']])


Epoch 1/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 461s 2s/step - accuracy: 0.6645 - loss: 0.8578 - val_accuracy: 0.7879 - val_loss: 0.5479
Epoch 2/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 567s 2s/step - accuracy: 0.8665 - loss: 0.3729 - val_accuracy: 0.8662 - val_loss: 0.3635
Epoch 3/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 463s 2s/step - accuracy: 0.9107 - loss: 0.2622 - val_accuracy: 0.9072 - val_loss: 0.2587
Epoch 4/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 441s 2s/step - accuracy: 0.9276 - loss: 0.2162 - val_accuracy: 0.8803 - val_loss: 0.3262
Epoch 5/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 441s 2s/step - accuracy: 0.9339 - loss: 0.1956 - val_accuracy: 0.8971 - val_loss: 0.2570
DenseNet121 Model Test Accuracy: 0.9068
              precision    recall  f1-score
Cyst           0.301513  0.349057  0.323548
Normal         0.412221  0.418307  0.415242
Stone          0.096899  0.090909  0.093809
Tumor          0.190058  0.142232  0.162703
accuracy       0.310843  0.310843  0.310843
macro avg      0.250173  0.250126  0.248825
weighted avg

In [ ]:
from tensorflow.keras.applications import DenseNet169

# Define and compile DenseNet-169 model
densenet169_base = DenseNet169(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
densenet169_base.trainable = False

densenet169_model = build_model(densenet169_base)

# Train the model
densenet169_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = densenet169_model.evaluate(test_generator, verbose=0)
y_true = test_generator.classes
y_pred = densenet169_model.predict(test_generator, verbose=0).argmax(axis=1)

# Classification metrics
report = classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys()), output_dict=True)
metrics_df = pd.DataFrame(report).transpose()
metrics_df['loss'] = test_loss

# Display results
print(f"DenseNet-169 Model Test Accuracy: {test_accuracy:.4f}")
print(metrics_df[['precision', 'recall', 'f1-score', 'support', 'loss']])


51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 31s 1us/step
Epoch 1/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 739s 3s/step - accuracy: 0.6578 - loss: 0.8826 - val_accuracy: 0.8421 - val_loss: 0.3980
Epoch 2/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 634s 3s/step - accuracy: 0.9012 - loss: 0.2929 - val_accuracy: 0.9024 - val_loss: 0.2744
Epoch 3/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 548s 2s/step - accuracy: 0.9376 - loss: 0.1942 - val_accuracy: 0.9321 - val_loss: 0.1896
Epoch 4/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 548s 2s/step - accuracy: 0.9417 - loss: 0.1766 - val_accuracy: 0.9345 - val_loss: 0.1706
Epoch 5/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 550s 2s/step - accuracy: 0.9578 - loss: 0.1307 - val_accuracy: 0.9385 - val_loss: 0.1634
DenseNet-169 Model Test Accuracy: 0.9450
              precision    recall  f1-score      support      loss
Cyst           0.280959  0.284367  0.282652   742.000000  0.142323
Normal         0.403458  0.413386  0.408362  1016.000000  0.142323
Stone          0.120130  0.134545  0.126930   275.000000  0.142323

In [ ]:
from tensorflow.keras.applications import DenseNet201

# Define and compile DenseNet-201 model
densenet201_base = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
densenet201_base.trainable = False

densenet201_model = build_model(densenet201_base)

# Train the model
densenet201_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = densenet201_model.evaluate(test_generator, verbose=0)
y_true = test_generator.classes
y_pred = densenet201_model.predict(test_generator, verbose=0).argmax(axis=1)

# Classification metrics
report = classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys()), output_dict=True)
metrics_df = pd.DataFrame(report).transpose()
metrics_df['loss'] = test_loss

# Display results
print(f"DenseNet-201 Model Test Accuracy: {test_accuracy:.4f}")
print(metrics_df[['precision', 'recall', 'f1-score', 'support', 'loss']])


74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 45s 1us/step
Epoch 1/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 888s 4s/step - accuracy: 0.6636 - loss: 0.8901 - val_accuracy: 0.8763 - val_loss: 0.3654
Epoch 2/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 848s 4s/step - accuracy: 0.8933 - loss: 0.3083 - val_accuracy: 0.9056 - val_loss: 0.2533
Epoch 3/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 751s 3s/step - accuracy: 0.9335 - loss: 0.2083 - val_accuracy: 0.9253 - val_loss: 0.2059
Epoch 4/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 746s 3s/step - accuracy: 0.9505 - loss: 0.1550 - val_accuracy: 0.9425 - val_loss: 0.1580
Epoch 5/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 756s 3s/step - accuracy: 0.9537 - loss: 0.1300 - val_accuracy: 0.9494 - val_loss: 0.1363
DenseNet-201 Model Test Accuracy: 0.9606
              precision    recall  f1-score      support      loss
Cyst           0.312343  0.334232  0.322917   742.000000  0.116252
Normal         0.411647  0.403543  0.407555  1016.000000  0.116252
Stone          0.110672  0.101818  0.106061   275.000000  0.116252